In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import os
import base64

def download_captcha_images(url, total_images, download_folder):
    options = webdriver.ChromeOptions()
    options.add_argument('--headless')
    driver = webdriver.Chrome(options=options)
    driver.get(url)
    
    if not os.path.exists(download_folder):
        os.makedirs(download_folder)

    for i in range(total_images):
        try:
            # Wait for the captcha image to be present
            captcha_img = WebDriverWait(driver, 20).until(
                EC.presence_of_element_located((By.XPATH, '//img[@alt="captcha"]'))
            )
            src = captcha_img.get_attribute('src')
            
            # Check if src is a base64 image
            if src.startswith('data:image'):
                base64_encoded_data = src.split(',')[1]
                image_data = base64.b64decode(base64_encoded_data)
                with open(os.path.join(download_folder, f'captcha_{i}.png'), 'wb') as file:
                    file.write(image_data)
            else:
                pass
            
            refresh_btn = WebDriverWait(driver, 20).until(
                EC.element_to_be_clickable((By.XPATH, '//img[@alt="refresh"]'))
            )
            refresh_btn.click()
            
            time.sleep(2)
            
        except Exception as e:
            print(f"An error occurred: {e}")
            break

    driver.quit()

url = "https://voters.eci.gov.in/download-eroll"
total_images = 5000
download_folder = './captchas_2'

download_captcha_images(url, total_images, download_folder)


KeyboardInterrupt: 